In [130]:
def refactor_monthly_data_frame(data_frame_location):
    print("Refactoring ", data_frame_location, " as per project requirement")
    import pandas as pd

    df = pd.read_csv(data_frame_location, sep=";")
    df["Credit"] = df["Credit"].astype(str).str.replace(",", ".").astype(float)
    df["Debit"] = df["Debit"].astype(str).str.replace(",", ".").astype(float)

    df = df.fillna(0)
    try:
        df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%y")
    except:
        pass

    df.to_csv(data_frame_location, sep=";", index=False)
    print("Refactoring of ", data_frame_location, " is completed !! ")
    return None

In [131]:
import pandas as pd

January_location = 'input_data/2023/01_Jan.csv'
February_location = 'input_data/2023/02_Feb.csv'
March_location = 'input_data/2023/03_March.csv'
April_location = 'input_data/2023/04_April.csv'
May_location = 'input_data/2023/05_May.csv'
June_location = 'input_data/2023/06_June.csv'
July_location = 'input_data/2023/07_July.csv'


for location in [January_location, February_location, March_location, April_location, May_location, June_location, July_location]:
    refactor_monthly_data_frame(location)


df_january = pd.read_csv(January_location, sep=';')
df_february = pd.read_csv(February_location, sep=';')
df_march = pd.read_csv(March_location, sep=';')
df_april = pd.read_csv(April_location, sep=';')
df_may = pd.read_csv(May_location, sep=';')
df_june = pd.read_csv(June_location, sep=';')
df_july = pd.read_csv(July_location, sep=';')

# concat all dataframes
df = pd.concat([df_january, df_february, df_march, df_april, df_may, df_june, df_july], ignore_index=True)

df["effective_amount"] = df["Credit"] - df["Debit"]
df["Balance_Amount"] = df["effective_amount"].cumsum()

# extract month from date column
df['Date'] = pd.to_datetime(df['Date'])
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month_name()
df['year'] = df['Date'].dt.year
df.drop(['Date'], axis=1, inplace=True)


df_category = pd.read_csv('input_data/category_mapper.csv', sep=';')

# add Category and Category Type column to df from df_category based on Sub-category
df['Category'] = df['Sub-category'].map(df_category.set_index('Sub-category')['Category'])
df['Category Type'] = df['Sub-category'].map(df_category.set_index('Sub-category')['Category Type'])
df.head(5)

Refactoring  input_data/2023/01_Jan.csv  as per project requirement
Refactoring of  input_data/2023/01_Jan.csv  is completed !! 
Refactoring  input_data/2023/02_Feb.csv  as per project requirement
Refactoring of  input_data/2023/02_Feb.csv  is completed !! 
Refactoring  input_data/2023/03_March.csv  as per project requirement
Refactoring of  input_data/2023/03_March.csv  is completed !! 
Refactoring  input_data/2023/04_April.csv  as per project requirement
Refactoring of  input_data/2023/04_April.csv  is completed !! 
Refactoring  input_data/2023/05_May.csv  as per project requirement
Refactoring of  input_data/2023/05_May.csv  is completed !! 
Refactoring  input_data/2023/06_June.csv  as per project requirement
Refactoring of  input_data/2023/06_June.csv  is completed !! 
Refactoring  input_data/2023/07_July.csv  as per project requirement
Refactoring of  input_data/2023/07_July.csv  is completed !! 


,Description,Debit,Credit,Sub-category,effective_amount,Balance_Amount,day,month,year,Category,Category Type
0,Old balance,0.00,8.02,Salary,8.02,8.02,27,January,2023,Salary,Income
1,LOHN / GEHALT 01/23,0.00,5417.00,Salary,5417.00,5425.02,27,January,2023,Salary,Income
2,Mobiko Brutto,0.00,75.00,Tax less Income,75.00,5500.02,27,January,2023,Extra Income,Income
3,Income Tax,1041.75,0.00,Tax,-1041.75,4458.27,27,January,2023,Income Tax,State Cuttings
4,Health Insurance,394.02,0.00,Health Insurance,-394.02,4064.25,27,January,2023,Health Insurance,State Cuttings


In [132]:
# pivot table of categorical variables for Debit 
import numpy as np
kpis_expenses = pd.pivot_table(df, index=['Category'], values=['Debit'], aggfunc=np.sum).reset_index()

kpis_to_india = kpis_expenses[kpis_expenses['Category'].isin(['India Savings'])]
kpis_to_india.columns = ['Category', 'amount']

kpis_to_eur_save = kpis_expenses[kpis_expenses['Category'].isin(['Europe Savings'])]
kpis_to_eur_save.columns = ['Category', 'amount']

kpis_expenses = kpis_expenses[~kpis_expenses['Category'].isin(['Salary', 'Extra Income', 'India Savings', 'Europe Savings'])]
kpis_expenses.columns = ['Category', 'amount']


kpis_income = pd.pivot_table(df, index=['Category'], values=['Credit'], aggfunc=np.sum).reset_index()
kpis_income = kpis_income[kpis_income['Category'].isin(['Salary', 'Extra Income'])]
kpis_income.columns = ['Category', 'amount']

df_KPIs = pd.concat([kpis_income, kpis_expenses, kpis_to_india, kpis_to_eur_save], axis=0)
df_KPIs.to_csv("intermediate_cache/KPIs.csv", sep=";", index=False)
df_KPIs

,Category,amount
3,Extra Income,1761.06
10,Salary,40169.02
0,Charity,1077.47
2,Extra Expenses,848.72
4,Health Insurance,2758.14
5,Income Tax,7570.55
7,Living Expenses,5935.67
8,PDP Expenses,654.64
9,Pension,3650.15
6,India Savings,17610.00


In [133]:
monthly_expenses = pd.pivot_table(df, index=['month','Category Type'], values=['Debit'], aggfunc=np.sum).reset_index()
monthly_expenses = monthly_expenses[monthly_expenses['Category Type'].isin(['Expense', 'State Cuttings', 'Savings'])]
monthly_expenses.columns = ['month','Category Type', 'amount']

monthly_income = pd.pivot_table(df, index=['month','Category Type'], values=['Credit'], aggfunc=np.sum).reset_index()
monthly_income = monthly_income[monthly_income['Category Type'].isin(['Income'])]
monthly_income.columns = ['month','Category Type', 'amount']


df_monthly_bars = pd.concat([monthly_income, monthly_expenses], axis=0)
df_monthly_bars.to_csv("intermediate_cache/monthly_bars.csv", sep=";", index=False)

df_monthly_bars

,month,Category Type,amount
8,January,Expense,0.00
10,January,Savings,3400.00
11,January,State Cuttings,2093.99
4,February,Expense,1775.52
6,February,Savings,1660.00
7,February,State Cuttings,2322.81
23,March,State Cuttings,2174.45
22,March,Savings,2500.00
20,March,Expense,1005.75
0,April,Expense,412.06


In [145]:
detailed_income = pd.pivot_table(df, index=['month','Sub-category'], values=['Credit'], aggfunc=np.sum).reset_index()
detailed_income = detailed_income[detailed_income['Sub-category'].isin(['Salary', 'Tax less Income', 'Office Travel Income'])]
detailed_income = detailed_income.rename(columns={'Credit':'amount', 'Sub-category':'category'})
detailed_income['amount'] = detailed_income['amount'].round(2)
detailed_income

,month,category,amount
10,April,Salary,5667.00
12,April,Tax less Income,243.80
20,February,Salary,5992.00
28,January,Salary,5425.02
30,January,Tax less Income,75.00
39,July,Office Travel Income,130.39
45,July,Salary,6084.00
47,July,Tax less Income,243.80
63,June,Salary,5667.00
65,June,Tax less Income,243.80


In [146]:
detailed_expenses = pd.pivot_table(df, index=['month','Sub-category'], values=['Debit'], aggfunc=np.sum).reset_index()
detailed_expenses = detailed_expenses[detailed_expenses['Sub-category'].isin(['Tax', 
                                                                        'Health Insurance', 
                                                                        'Pension'
                                                                        'Unemployment Fund',
                                                                        'Nursing Care',
                                                                        'Rent',
                                                                        'Phone & WiFi',
                                                                        'Groceries',
                                                                        'Travel Pass',
                                                                        'Gifts',
                                                                        'Entertainment',
                                                                        'Furnishings',
                                                                        'Family & Kids',
                                                                        'Restaurant',
                                                                        'Taxi',
                                                                        'Cloths',
                                                                        'Gym & Self grooming',
                                                                        'PDP Spending',
                                                                        'Office Travel Spending',
                                                                        'To India',
                                                                        'Company Pension'
                                                                        ])]
detailed_expenses = detailed_expenses.rename(columns={'Debit':'amount', 'Sub-category':'category'})
detailed_expenses['amount'] = detailed_expenses['amount'].round(2)
detailed_expenses

,month,category,amount
0,April,Company Pension,292.00
1,April,Gifts,5.09
2,April,Groceries,134.11
3,April,Gym & Self grooming,20.00
4,April,Health Insurance,394.02
...,...,...,...
91,May,PDP Spending,109.99
93,May,Rent,590.00
95,May,Tax,1015.50
97,May,To India,2600.00


In [148]:
df_detailed_values = pd.concat([detailed_income, detailed_expenses], axis=0)
df_detailed_values.to_csv("intermediate_cache/detailed_values.csv", sep=";", index=False)
df_detailed_values

,month,category,amount
10,April,Salary,5667.00
12,April,Tax less Income,243.80
20,February,Salary,5992.00
28,January,Salary,5425.02
30,January,Tax less Income,75.00
...,...,...,...
91,May,PDP Spending,109.99
93,May,Rent,590.00
95,May,Tax,1015.50
97,May,To India,2600.00


In [111]:
df_category

,Sub-category,Category,Category Type
0,Salary,Salary,Income
1,Tax less Income,Extra Income,Income
2,PDP Income,Extra Income,Income
3,Office Travel Income,Extra Income,Income
4,Tax,Income Tax,State Cuttings
5,Health Insurance,Health Insurance,State Cuttings
6,Pension,Pension,State Cuttings
7,Unemployment Fund,Charity,State Cuttings
8,Nursing Care,Charity,State Cuttings
9,Rent,Living Expenses,Expense
